In [ ]:
import multiModel as mm
from multiModel import MultiInputModel
from glob import glob
import os
import matplotlib.pyplot as plt
import numpy as np

# Załaduj dane
test_dataset = mm.MultiInputDataset("CSV/dataset/test.csv", transform_rgb=mm.transform_rgb, transform_binary=mm.transform_binary)
test_loader = mm.DataLoader(test_dataset, batch_size=32, shuffle=False)

for path_model in glob("training_results/*.pth"):
    # Wywołanie funkcji testującej
    cm, y_true, y_pred = mm.test_model(
        path_model=path_model,
        test_loader=test_loader,
        device="cuda"
    )

    # Wyświetlenie macierzy pomyłek
    class_names = list(test_dataset.class_to_idx.keys())
    plt.figure(figsize=(10, 8))
    mm.sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Przewidywane klasy")
    plt.ylabel("Rzeczywiste klasy")
    plt.title(f"Macierz pomyłek: {os.path.split(os.path.basename(path_model))[0]}")
    plt.savefig(os.path.splitext(path_model)[0] + ".png", dpi=300, bbox_inches="tight")
    plt.show()

    # Inne metryki


    # Otwórz plik w trybie zapisu
    with open(os.path.splitext(path_model)[0] + ".txt", "w") as file:
        # Zapisz dokładność
        accuracy = mm.accuracy_score(y_true, y_pred)
        file.write(f"Dokładność: {accuracy:.4f}\n")
        
        # Zapisz raport klasyfikacji
        classification_report = mm.classification_report(y_true, y_pred, target_names=class_names)
        file.write("\nRaport klasyfikacji:\n")
        file.write(classification_report + "\n")
        
        # Zapisz informacje o największej liczbie pomyłek
        off_diagonal = cm - np.diag(np.diag(cm))  # Macierz bez przekątnej
        max_misclassified = np.unravel_index(np.argmax(off_diagonal), off_diagonal.shape)
        
        most_confused_classes = f"Najczęściej mylone klasy: {class_names[max_misclassified[0]]} → {class_names[max_misclassified[1]]}"
        misclassified_count = f"Liczba pomyłek: {off_diagonal[max_misclassified]}"
        
        file.write("\n" + most_confused_classes + "\n")
        file.write(misclassified_count + "\n")
        file.close()

    # Wyświetl w konsoli również, jeśli chcesz
    print(f"Dokładność: {accuracy:.4f}")
    print("\nRaport klasyfikacji:")
    print(classification_report)
    print(most_confused_classes)
    print(misclassified_count)